# Niwot and Sod Scatter Plots 1
* looking at the wind classification for Niwot and Sodankyla 
* looking at the "sublimation" classification for Niwot (only)

## Import packages 

In [1]:
# import packages 
%matplotlib widget
# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

In [2]:
import seaborn as sns # a module that adds some plotting capabilities and makes your plots look better
import matplotlib as mpl
sns.set() # activates some of the default settings from seaborn
# The following settings just set some defaults for the plots
plt.rcParams['figure.figsize']  = (12,4) #width, height
plt.rcParams['axes.titlesize']  = 14
plt.rcParams['axes.labelsize']  = 12
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 11
mpl.rcParams['figure.dpi'] = 100

sns.set_style("dark", {"xtick.bottom": True, 'ytick.left': True})

## Load and Clean datasets for use

### Niwot

In [3]:
# AMERIFLUX MET DATA 
niwotflux =   pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/niwot_2017_ameriflux_unload1hr.csv")
niwotflux['datetime']  = pd.DatetimeIndex(niwotflux['datetime'])

# OBSERVATIONS CLASSIFICATIONS
niwotobs =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_niwot2017_type.csv")
niwotobs['datetime'] = pd.to_datetime(niwotobs['datetime'])

niwotobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERG THEM 
niwotdf = pd.merge(niwotflux, niwotobs, how='outer', on='datetime')
niwotdf.index = pd.DatetimeIndex(niwotdf['datetime'])

In [4]:
# OPEN MET DATA 
sod =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Sodankyla/SOD_1819_1hr_cleaned.csv")
sod['datetime'] = pd.to_datetime(sod['datetime'])

# OPEN CLASSIFICATIONS
sodobs = pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Classifications/datetimeformat_classifications_sodankyla2019_type.csv")
sodobs['datetime'] = pd.to_datetime(sodobs['datetime'])
sodobs.dropna(axis=0, how='all', inplace=True) #removing row is entire row is NAN #careful with this, without datetime a lot gets removed 

# MERGE
soddf = pd.merge(sod, sodobs, how='outer', on='datetime')
soddf.index = pd.DatetimeIndex(soddf['datetime'])
soddf.drop(columns=['datetime'], inplace=True)

# # Create seperate df for only snow in the canopy timesteps 
# sod_df_unload = soddf.copy(deep=True)
# sod_df_unload = sod_df_unload.dropna(axis=0, subset=['CD'])

# # Have to remove all nans for this plotting to work.... need to come back to this 
# # sod_df_unload.dropna(inplace=True)

# # Create sunlit column
# sod_df_unload['Sunlit'] = sod_df_unload.E.copy(deep=True)
# sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
# sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# # Create unloading classification column
# sod_df_unload['Classification'] = sod_df_unload.CD.copy(deep=True)
# sod_df_unload.Classification.mask(sod_df_unload.Classification == 0, "Snow Unloading", inplace=True)
# sod_df_unload.Classification.mask(sod_df_unload.Classification == 1, "Snow Staying", inplace=True)

# sod_df_snowstay   = sod_df_unload.where(sod_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
# sod_df_snowunload = sod_df_unload.where(sod_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

#### Making "type" a symbol too
**Type**
* type 1 = some unloading from the event 
* type 2 = all unloaded from the event

**Snow**
* snow sub: sublimation
* snow mass: mass unloading
* snow melt: melt unloading

In [5]:
# https://plotly.com/python/marker-style
# SUNLIT-------------------------------------------------------
# Create symbols for sunlit or not
niwotdf['Esymbol'] = np.nan

# open triangle for cloudy, diamond for sunny, open x circle for no radiation data
niwotdf['Esymbol'].mask(niwotdf['E'] == 0., 105, inplace=True)
niwotdf['Esymbol'].mask(niwotdf['E'] == 1., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwotdf['Esymbol'] = niwotdf['Esymbol'].fillna(128) 

# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
niwotdf['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
niwotdf['Typesymbol'].mask(niwotdf['Type'] == 1, 102, inplace=True) 
niwotdf['Typesymbol'].mask(niwotdf['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwotdf['Typesymbol'] = niwotdf['Typesymbol'].fillna(128)  

# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
niwotdf['Snowsymbol'] = np.nan

# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'sub', 17, inplace=True)  # Full Star
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwotdf['Snowsymbol'] = niwotdf['Snowsymbol'].fillna(100)  # empty circle

#_________________________________________________________

# Create seperate df for only snow in the canopy timesteps 
niwot_df_unload = niwotdf.copy(deep=True)
niwot_df_unload = niwot_df_unload.dropna(axis=0, subset=['CD'])

# Have to remove all nans for this plotting to work.... need to come back to this 
# df_unload.dropna(inplace=True) #########KEEP THIS IN MIND, COME BACK TO IT

# Create sunlit column
niwot_df_unload['Sunlit'] = niwot_df_unload.E.copy(deep=True)
niwot_df_unload.Sunlit.mask(niwot_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
niwot_df_unload.Sunlit.mask(niwot_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
niwot_df_unload['Classification'] = niwot_df_unload.CD.copy(deep=True)
niwot_df_unload.Classification.mask(niwot_df_unload.Classification == 0, "Snow Unloading", inplace=True)
niwot_df_unload.Classification.mask(niwot_df_unload.Classification == 1, "Snow Staying in the Canopy", inplace=True)

niwot_df_snowstay   = niwot_df_unload.where(niwot_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
niwot_df_snowunload = niwot_df_unload.where(niwot_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

### Sodankyla

In [6]:
# https://plotly.com/python/marker-style
# SUNLIT-------------------------------------------------------
# Create symbols for sunlit or not
soddf['Esymbol'] = np.nan

# open triangle for cloudy, diamond for sunny, open x circle for no radiation data
soddf['Esymbol'].mask(soddf['E'] == 0., 105, inplace=True)
soddf['Esymbol'].mask(soddf['E'] == 1., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
soddf['Esymbol'] = soddf['Esymbol'].fillna(128) 

# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
soddf['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
soddf['Typesymbol'].mask(soddf['Type'] == 1, 102, inplace=True) 
soddf['Typesymbol'].mask(soddf['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
soddf['Typesymbol'] = soddf['Typesymbol'].fillna(128)  

# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
soddf['Snowsymbol'] = np.nan

# soddf['Snowsymbol'].mask(soddf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
soddf['Snowsymbol'].mask(soddf['Snow'] == 'sub', 17, inplace=True)  # Full Star
# soddf['Snowsymbol'].mask(soddf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
soddf['Snowsymbol'] = soddf['Snowsymbol'].fillna(100)  # empty circle

#_________________________________________________________

# Create seperate df for only snow in the canopy timesteps 
sod_df_unload = soddf.copy(deep=True)
sod_df_unload = sod_df_unload.dropna(axis=0, subset=['CD'])

# Have to remove all nans for this plotting to work.... need to come back to this 
# df_unload.dropna(inplace=True) #########KEEP THIS IN MIND, COME BACK TO IT

# Create sunlit column
sod_df_unload['Sunlit'] = sod_df_unload.E.copy(deep=True)
sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 0, "Not Sunlit", inplace=True)
sod_df_unload.Sunlit.mask(sod_df_unload.Sunlit == 1, "Sunlit Canopy", inplace=True)

# Create unloading classification column
sod_df_unload['Classification'] = sod_df_unload.CD.copy(deep=True)
sod_df_unload.Classification.mask(sod_df_unload.Classification == 0, "Snow Unloading", inplace=True)
sod_df_unload.Classification.mask(sod_df_unload.Classification == 1, "Snow Staying in the Canopy", inplace=True)

sod_df_snowstay   = sod_df_unload.where(sod_df_unload.CD == 1).dropna(axis=0, subset=['CD']) # where CD == 1, meaning Snow Staying, make that snowstaydf
sod_df_snowunload = sod_df_unload.where(sod_df_unload.CD == 0).dropna(axis=0, subset=['CD']) # where CD == 0, meaning Snow Unloading, make that snowunloaddf

# Plotting

In [7]:
## Define some plotting constants for easier coding 
plt.close('all')

# Colors
colornosnow='lightgrey' # the plotly no snow color so that it is lightly and not dominate in the plot 
colorsnow = '#7dcfd4' # slightly desaturated cyan
colorsnowunload = '#1F15D5' # bright, deep blue 
colorsunny = '#E4E44A' # trying a little less bright 

# Grouped Colors
colors = [colorsnow, colorsnowunload] # my colors 

# Names
namesnow = 'Snow Staying'
namesnowunload = 'Snow Unloading'
group_labels = ['Snow Staying', 'Snow Unloading']

nametemp = "Air Temperature (C)"
nameSW = "Shortwave (W/m2)"
namewind = "Wind Speed (m/s)"

# Hist defaults
bintemp = [2,2]
binSW = [100,100]
binwind = [1,1]

In [8]:
## Define some plotting constants for easier coding 
plt.close('all')

# Colors
colornosnow = '#D2B48C' # nice tan
colorsnow = '#7dcfd4' # slightly desaturated cyan
colorsnowunload = '#1F15D5' # bright, deep blue 
colorsunny = '#E4E44A' # trying a little less bright 
unloadcolors = [colorsnow, colorsnowunload]

# Names
namesnow = 'Snow Staying'
namesnowunload = 'Snow Unloading'
group_labels = ['Snow Staying', 'Snow Unloading']

nametemp = "Air Temperature (C)"
namewind = "Wind Speed (m/s)"
nameSW = "Shortwave (W/m2)"

# namelaret = 'Laret, Switzerland'
# nameniwot = 'Niwot Ridge, Colorado, USA'
# namehopper = 'Mount Hopper, Washington, USA'
# namesodankyla = 'Sodankyla, Finland'

namesodankyla = 'A)  Sodankyla, Finland'
namesod = 'A)  Sodankyla, Finland'
namelaret = 'B)  Davos Laret, Switzerland'
nameniwot = 'C)  Niwot Ridge, Colorado, USA'

# Scatter Plots: for partial unloading

In [9]:
# State which site we are working with 
df = niwot_df_unload.copy(deep=True)

# Some constatns 
colorscaled = [[0., colorsnowunload], [1, colorsnow]]
index_vals = df['CD']
index_symbols = df['Typesymbol']

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[dict(label='Air Temperature (C)',     values=df['temp']),
                                   dict(label='Wind Speed (m/s)', values=df['windspeed']),
                                   dict(label='Shortwave (W/m2)', values=df['shortwave'])],
                                   marker=dict(color=index_vals, colorscale=colorscaled, symbol=index_symbols, showscale=False, line_color='black', line_width=0.5, size=7),
                                   showlegend=False, showupperhalf=False, diagonal_visible=False,
                      ))

# Other options
#showupperhalf=False, diagonal_visible=False
#marker=dict(show scale=False)

# fig.update_traces(showlegend=True, legendgroup=index_symbols, selector=dict(mode='spolm'))
# fig.update_traces(showlegend=True)

fig.update_layout(title='Niwot Ridge, Colorado, USA', width=800, height=700, dragmode='select')

fig.show()

## Looking at Shapped by Sublimation Obs 

Niwot first

In [10]:
niwot_df_sub = niwot_df_unload.copy(deep=True)

# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
niwot_df_sub['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
niwot_df_sub['Typesymbol'].mask(niwot_df_sub['Type'] == 1, 102, inplace=True) 
niwot_df_sub['Typesymbol'].mask(niwot_df_sub['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwot_df_sub['Typesymbol'] = niwot_df_sub['Typesymbol'].fillna(128)  

# Class-------------------------------------------------------
# Create symbols for sunlit or not
niwot_df_sub['Classsymbol'] = np.nan

niwot_df_sub['Classsymbol'].mask(niwot_df_sub['Class'] == 'wind', 17, inplace=True)  # Full Star
niwot_df_sub['Classsymbol'].mask(niwot_df_sub['Class'] == 'tempwind', 17, inplace=True)  # Full Star
niwot_df_sub['Classsymbol'].mask(niwot_df_sub['Class'] == 'swwind', 17, inplace=True)  # Full Star
niwot_df_sub['Classsymbol'].mask(niwot_df_sub['Class'] == 'tempswwind', 17, inplace=True)  # Full Star


# then fillna with 128 for circle with x through it 
niwot_df_sub['Classsymbol'] = niwot_df_sub['Classsymbol'].fillna(128)  # empty circle


# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
niwot_df_sub['Snowsymbol'] = np.nan

# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
niwot_df_sub['Snowsymbol'].mask(niwot_df_sub['Snow'] == 'sub', 17, inplace=True)  # Full Star
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
niwot_df_sub['Snowsymbol'] = niwot_df_sub['Snowsymbol'].fillna(128)  # empty circle

In [11]:
niwot_df_sub.head()

,datetime,shortwave,longwave,precip,windspeed,RH,temp,A,B,CD,E,Type,Class,Snow,Esymbol,Typesymbol,Snowsymbol,Sunlit,Classification,Classsymbol
datetime,,,,,,,,,,,,,,,,,,,,
2016-10-06 06:00:00,2016-10-06 06:00:00,4.93,297.470,0.000,1.180,96.650,-3.495,1.0,0.0,1.0,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2016-10-06 07:00:00,2016-10-06 07:00:00,29.43,298.655,0.000,1.735,93.195,-3.325,1.0,0.0,1.0,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2016-10-06 08:00:00,2016-10-06 08:00:00,124.50,309.965,0.225,2.080,83.190,-2.430,1.0,0.0,1.0,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2016-10-06 09:00:00,2016-10-06 09:00:00,102.04,306.465,0.700,2.745,90.135,-3.250,1.0,0.0,1.0,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2016-10-06 10:00:00,2016-10-06 10:00:00,614.38,240.145,0.075,4.595,80.390,-3.350,1.0,1.0,1.0,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0


Now Sodankyla with the same code

In [12]:
sod_df_sub = sod_df_unload.copy(deep=True)

# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
sod_df_sub['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
sod_df_sub['Typesymbol'].mask(sod_df_sub['Type'] == 1, 102, inplace=True) 
sod_df_sub['Typesymbol'].mask(sod_df_sub['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_sub['Typesymbol'] = sod_df_sub['Typesymbol'].fillna(128)  

# Class-------------------------------------------------------
# Create symbols for sunlit or not
sod_df_sub['Classsymbol'] = np.nan

sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'wind', 17, inplace=True)  # Full Star
sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'tempwind', 17, inplace=True)  # Full Star
sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'swwind', 17, inplace=True)  # Full Star
sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'tempswwind', 17, inplace=True)  # Full Star


# then fillna with 128 for circle with x through it 
sod_df_sub['Classsymbol'] = sod_df_sub['Classsymbol'].fillna(128)  # empty circle


# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
sod_df_sub['Snowsymbol'] = np.nan

# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
sod_df_sub['Snowsymbol'].mask(sod_df_sub['Snow'] == 'sub', 17, inplace=True)  # Full Star
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_sub['Snowsymbol'] = sod_df_sub['Snowsymbol'].fillna(128)  # empty circle

In [13]:
sod_df_sub.head()

,shortwave,longwave,ppt_1,ppt1,RH,windspeed,pressure,snowdepth,temp,A,...,E,Type,Class,Snow,Esymbol,Typesymbol,Snowsymbol,Sunlit,Classification,Classsymbol
datetime,,,,,,,,,,,,,,,,,,,,,
2018-10-05 14:00:00,36.25,312.55,0.000379,1.630000e-12,98.0,3.61035,96730.0,7.0,0.15,1.0,...,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2018-10-05 15:00:00,39.60,312.30,0.000217,1.300000e-12,97.5,2.38995,96635.0,8.0,0.20,1.0,...,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2018-10-05 16:00:00,23.25,312.05,0.000126,5.450000e-13,97.0,1.27125,96595.0,7.0,0.15,1.0,...,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2018-10-05 17:00:00,2.10,311.30,0.000054,1.200000e-13,97.5,0.76275,96610.0,9.0,0.05,1.0,...,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0
2018-10-05 18:00:00,0.00,309.70,0.000072,8.200000e-14,98.0,1.27125,96650.0,9.0,-0.05,1.0,...,0.0,NaN,NaN,NaN,105.0,128.0,128.0,Not Sunlit,Snow Staying in the Canopy,128.0


In [14]:
# Niwot
colorscaled = [[0., colorsnowunload], [1, colorsnow]]
index_vals = niwot_df_sub['CD']
index_symbols = niwot_df_sub['Snowsymbol']

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[dict(label=nametemp, values=niwot_df_sub['temp']),
                                   dict(label=namewind, values=niwot_df_sub['windspeed']),
                                   dict(label=nameSW,   values=niwot_df_sub['shortwave'])],
                                   marker=dict(color=index_vals, colorscale=colorscaled, symbol=index_symbols, showscale=False, line_color='black', line_width=0.5, size=7),
                                   showlegend=False, showupperhalf=False, diagonal_visible=False,
                      ))

fig.update_layout(title='Niwot Ridge, Colorado, USA: star = snow blowing around the trees', width=800, height=700, dragmode='select')

fig.show()

In [15]:
# Niwot
# Some constatns 
colorscaled = [[0., colorsnowunload], [1, colorsnow]]
index_vals = niwot_df_sub['CD']
index_symbols = niwot_df_sub['Classsymbol']

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[dict(label=nametemp, values=niwot_df_sub['temp']),
                                   dict(label=namewind, values=niwot_df_sub['windspeed']),
                                   dict(label=nameSW,   values=niwot_df_sub['shortwave'])],
                                   marker=dict(color=index_vals, colorscale=colorscaled, symbol=index_symbols, showscale=False, line_color='black', line_width=0.5, size=7),
                                   showlegend=False, showupperhalf=False, diagonal_visible=False,
                      ))

fig.update_layout(title='Niwot Ridge, Colorado, USA: star = trees moving in the wind', width=800, height=700, dragmode='select')

fig.show()

## Now for Sodankyla

In [16]:
fig = go.Figure(data=go.Scatter(
    x = niwot_df_sub['temp'],
    y = niwot_df_sub['shortwave'],
    mode='markers',
    marker=dict(
        size=16,
        color=niwot_df_sub['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub['Classsymbol'],
        showscale=False
    )
))

fig.update_layout(title='Niwot Ridge, Colorado, USA', width=800, height=700, dragmode='select')
fig.update_xaxes(range=[-30,10]) # temp
fig.update_yaxes(range=[-50,1200]) # shortwave

fig.show()

In [17]:
# Sodankyla 
# Some constatns 
colorscaled = [[0., colorsnowunload], [1, colorsnow]]
index_vals = sod_df_sub['CD']
index_symbols = sod_df_sub['Classsymbol']

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[dict(label=nametemp, values=sod_df_sub['temp']),
                                   dict(label=namewind, values=sod_df_sub['windspeed']),
                                   dict(label=nameSW,   values=sod_df_sub['shortwave'])],
                                   marker=dict(color=index_vals, colorscale=colorscaled, symbol=index_symbols, showscale=False, line_color='black', line_width=0.5, size=7),
                                   showlegend=False, showupperhalf=False, diagonal_visible=False,
                      ))

fig.update_layout(title='Sodankyla, Findland: star = trees moving in the wind', width=800, height=700, dragmode='select')

fig.show()

## Together

In [18]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=(namesod, '', nameniwot), 
                    shared_xaxes=False, shared_yaxes=False, 
                    vertical_spacing=0.08, horizontal_spacing=0.1,
)

markersize = 8

# Sodankyla (only temp x wind)
fig.add_trace(
    go.Scatter(
        x = sod_df_sub['temp'],
        y = sod_df_sub['windspeed'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=sod_df_sub['CD'], 
        colorscale=colorscaled, 
        symbol=sod_df_sub['Classsymbol'],
        showscale=False,
        )
    ), row=1, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-35,8], row=1, col=1) 
fig.update_yaxes(title_text = namewind, range=[-.5,15], row=1, col=1) 


# Niwot, full scatter matrix 
fig.add_trace(
    go.Scatter(
        x = niwot_df_sub['temp'],
        y = niwot_df_sub['windspeed'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=niwot_df_sub['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub['Classsymbol'],
        showscale=False
        )
    ), row=2, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-35,8], row=2, col=1) 
fig.update_yaxes(title_text = namewind, range=[-.5,15], row=2, col=1) 

fig.add_trace(
    go.Scatter(
        x = niwot_df_sub['temp'],
        y = niwot_df_sub['shortwave'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=niwot_df_sub['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub['Classsymbol'],
        showscale=False
        )
    ), row=3, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-35,8], row=3, col=1) 
fig.update_yaxes(title_text = nameSW, range=[-50,1200], row=3, col=1)

fig.add_trace(
    go.Scatter(
        x = niwot_df_sub['windspeed'],
        y = niwot_df_sub['shortwave'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=niwot_df_sub['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub['Classsymbol'],
        showscale=False,
        )
    ), row=3, col=2
)

fig.update_xaxes(title_text = namewind, range=[-.5,15], row=3, col=2) 
fig.update_yaxes(title_text = nameSW, range=[-50,1200], row=3, col=2)

fig.update_layout(title='', width=1000, height=1200, showlegend=False)
fig.show()

## Now, we want to plot the trees moving in the wind and the partial unloading together in one plot
**legend** *observation: symbol-name (#)* 
* snow staying: circle-x-open (128)
* partial unloading: diamond-open (102)
* full unloading: diamond (2)

* partial unloading && tree moving: star-open (117)
* full unloading && tree moving: star (17)

In [19]:
sod_df_sub = sod_df_unload.copy(deep=True)

# TYPE-------------------------------------------------------
# Create symbols for type 1 and type 2 
sod_df_sub['Typesymbol'] = np.nan

# open diamont for type 1, diamond for type 2, open x circle for no classification
sod_df_sub['Typesymbol'].mask(sod_df_sub['Type'] == 1, 102, inplace=True) 
sod_df_sub['Typesymbol'].mask(sod_df_sub['Type'] == 2., 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_sub['Typesymbol'] = sod_df_sub['Typesymbol'].fillna(128)  

# Class-------------------------------------------------------
# Create symbols for sunlit or not
sod_df_sub['Classsymbol'] = np.nan

sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'wind', 17, inplace=True)  # Full Star
sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'tempwind', 17, inplace=True)  # Full Star
sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'swwind', 17, inplace=True)  # Full Star
sod_df_sub['Classsymbol'].mask(sod_df_sub['Class'] == 'tempswwind', 17, inplace=True)  # Full Star


# then fillna with 128 for circle with x through it 
sod_df_sub['Classsymbol'] = sod_df_sub['Classsymbol'].fillna(128)  # empty circle


# SNOW-------------------------------------------------------
# Create symbols for sunlit or not
sod_df_sub['Snowsymbol'] = np.nan

# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'mass', 105, inplace=True) # this is working without fillna
sod_df_sub['Snowsymbol'].mask(sod_df_sub['Snow'] == 'sub', 17, inplace=True)  # Full Star
# niwotdf['Snowsymbol'].mask(niwotdf['Snow'] == 'melt', 2, inplace=True)    

# then fillna with 128 for circle with x through it 
sod_df_sub['Snowsymbol'] = sod_df_sub['Snowsymbol'].fillna(128)  # empty circle

In [20]:
# Niwot 
# create a new column for the partial and wind classification varible
niwot_df_sub2 = niwot_df_sub.copy(deep=True)

# first, copy type into the new column
niwot_df_sub2['TypeClassSymbol'] = niwot_df_sub2['Typesymbol'].copy(deep=True) 

# then, change the values when Type == 1 AND Class == 17, then it's an open star for wind and partial unloading
niwot_df_sub2['TypeClassSymbol'].mask((niwot_df_sub2['Typesymbol'] == 102) & (niwot_df_sub2['Classsymbol'] == 17), 117, inplace  = True) # open star 

# last, change the values when Type == 2 AND Class == 17, then it's an closed star for wind and full unloading
niwot_df_sub2['TypeClassSymbol'].mask((niwot_df_sub2['Typesymbol'] == 2) & (niwot_df_sub2['Classsymbol'] == 17), 17, inplace  = True) # closed star 

# OPTION 1: last, fill all the snow staying with the circle
niwot_df_sub2['TypeClassSymbol'] = niwot_df_sub2['TypeClassSymbol'].fillna(128)  # empty circle

# OPTION 2: unless you do not want the snow staying to plot then remove everything else 
# niwot_df_sub2.dropna(subset=['TypeClassSymbol'], inplace=True)


# Sodankyla 
# create a new column for the partial and wind classification varible
sod_df_sub2 = sod_df_sub.copy(deep=True)

# first, copy type into the new column
sod_df_sub2['TypeClassSymbol'] = sod_df_sub2['Typesymbol'].copy(deep=True) 

# then, change the values when Type == 1 AND Class == 17, then it's an open star for wind and partial unloading
sod_df_sub2['TypeClassSymbol'].mask((sod_df_sub2['Typesymbol'] == 102) & (sod_df_sub2['Classsymbol'] == 17), 117, inplace  = True) # open star 

# last, change the values when Type == 2 AND Class == 17, then it's an closed star for wind and full unloading
sod_df_sub2['TypeClassSymbol'].mask((sod_df_sub2['Typesymbol'] == 2) & (sod_df_sub2['Classsymbol'] == 17), 17, inplace  = True) # closed star

# OPTION 1: last, fill all the snow staying with the circle 
sod_df_sub2['TypeClassSymbol'] = sod_df_sub2['TypeClassSymbol'].fillna(128)  # empty circle

# OPTION 2: unless you do not want the snow staying to plot then remove everything else 
# sod_df_sub2.dropna(subset=['TypeClassSymbol'], inplace=True)


In [25]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=(namesod, '', "B)  Niwot Ridge, Colorado, USA"), 
                    shared_xaxes=False, shared_yaxes=False, 
                    vertical_spacing=0.08, horizontal_spacing=0.1,
)

markersize = 8

# Sodankyla (only temp x wind)
fig.add_trace(
    go.Scatter(
        x = sod_df_sub2['temp'],
        y = sod_df_sub2['windspeed'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=sod_df_sub2['CD'], 
        colorscale=colorscaled, 
        symbol=sod_df_sub2['TypeClassSymbol'],
        showscale=False,
        )
    ), row=1, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-35,8], row=1, col=1) 
fig.update_yaxes(title_text = namewind, range=[-.5,15], row=1, col=1) 


# Niwot, full scatter matrix 
fig.add_trace(
    go.Scatter(
        x = niwot_df_sub2['temp'],
        y = niwot_df_sub2['windspeed'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=niwot_df_sub2['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub2['TypeClassSymbol'],
        showscale=False
        )
    ), row=2, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-35,8], row=2, col=1) 
fig.update_yaxes(title_text = namewind, range=[-.5,15], row=2, col=1) 

fig.add_trace(
    go.Scatter(
        x = niwot_df_sub2['temp'],
        y = niwot_df_sub2['shortwave'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=niwot_df_sub2['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub2['TypeClassSymbol'],
        showscale=False
        )
    ), row=3, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-35,8], row=3, col=1) 
fig.update_yaxes(title_text = nameSW, range=[-50,1200], row=3, col=1)

fig.add_trace(
    go.Scatter(
        x = niwot_df_sub2['windspeed'],
        y = niwot_df_sub2['shortwave'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=niwot_df_sub2['CD'], 
        colorscale=colorscaled, 
        symbol=niwot_df_sub2['TypeClassSymbol'],
        showscale=False,
        )
    ), row=3, col=2
)

fig.update_xaxes(title_text = namewind, range=[-.5,15], row=3, col=2) 
fig.update_yaxes(title_text = nameSW, range=[-50,1200], row=3, col=2)

fig.update_layout(title='', width=1000, height=1200, showlegend=False)
fig.show()

## Remove the snow staying, and only keep the snow unloading

In [21]:
niwot_df_sub3.head()

NameError: name 'niwot_df_sub3' is not defined

In [22]:
# Only save the unloading events information

# Niwot 
# create a new column for the partial and wind classification varible
niwot_df_sub3 = niwot_df_sub.copy(deep=True)
niwot_df_sub3['CD'] = niwot_df_sub3['CD'].where(niwot_df_sub3['CD'] == 0).dropna()
niwot_df_sub3['CD'].dropna(inplace=True)

# first, copy type into the new column
niwot_df_sub3['TypeClassSymbol'] = niwot_df_sub3['Typesymbol'].copy(deep=True) 

# then, change the values when Type == 1 AND Class == 17, then it's an open star for wind and partial unloading
niwot_df_sub3['TypeClassSymbol'].mask((niwot_df_sub3['Typesymbol'] == 102) & (niwot_df_sub3['Classsymbol'] == 17), 117, inplace  = True) # open star 

# last, change the values when Type == 2 AND Class == 17, then it's an closed star for wind and full unloading
niwot_df_sub3['TypeClassSymbol'].mask((niwot_df_sub3['Typesymbol'] == 2) & (niwot_df_sub3['Classsymbol'] == 17), 17, inplace  = True) # closed star 

# OPTION 1: last, fill all the snow staying with the circle
# niwot_df_sub3['TypeClassSymbol'] = niwot_df_sub3['TypeClassSymbol'].fillna(128)  # empty circle

# OPTION 2: unless you do not want the snow staying to plot then remove everything else 
niwot_df_sub3.dropna(subset=['TypeClassSymbol'], inplace=True)
niwot_df_sub3.dropna(subset=['CD'], inplace=True)


# Sodankyla 
# create a new column for the partial and wind classification varible
sod_df_sub3 = sod_df_sub.copy(deep=True)
sod_df_sub3['CD'] = sod_df_sub3['CD'].where(sod_df_sub3['CD'] == 0).dropna()
sod_df_sub3['CD'].dropna(inplace=True)

# first, copy type into the new column
sod_df_sub3['TypeClassSymbol'] = sod_df_sub3['Typesymbol'].copy(deep=True) 

# then, change the values when Type == 1 AND Class == 17, then it's an open star for wind and partial unloading
sod_df_sub3['TypeClassSymbol'].mask((sod_df_sub3['Typesymbol'] == 102) & (sod_df_sub3['Classsymbol'] == 17), 117, inplace  = True) # open star 

# last, change the values when Type == 2 AND Class == 17, then it's an closed star for wind and full unloading
sod_df_sub3['TypeClassSymbol'].mask((sod_df_sub3['Typesymbol'] == 2) & (sod_df_sub3['Classsymbol'] == 17), 17, inplace  = True) # closed star

# OPTION 1: last, fill all the snow staying with the circle 
# sod_df_sub3['TypeClassSymbol'] = sod_df_sub3['TypeClassSymbol'].fillna(128)  # empty circle

# OPTION 2: unless you do not want the snow staying to plot then remove everything else 
sod_df_sub3.dropna(subset=['TypeClassSymbol'], inplace=True)
sod_df_sub3.dropna(subset=['CD'], inplace=True)

In [23]:
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=(namesod, '', "B)  Niwot Ridge, Colorado, USA"), 
                    shared_xaxes=False, shared_yaxes=False, 
                    vertical_spacing=0.08, horizontal_spacing=0.1,
)

markersize = 8

# Sodankyla (only temp x wind)
fig.add_trace(
    go.Scatter(
        x = sod_df_sub3['temp'],
        y = sod_df_sub3['windspeed'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=colorsnowunload,
        colorscale=colorscaled, 
        symbol=sod_df_sub3['TypeClassSymbol'],
        showscale=False,
        )
    ), row=1, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-22,8], row=1, col=1) 
fig.update_yaxes(title_text = namewind, range=[-.5,15], row=1, col=1) 


# Niwot, full scatter matrix 
fig.add_trace(
    go.Scatter(
        x = niwot_df_sub3['temp'],
        y = niwot_df_sub3['windspeed'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=colorsnowunload,
        colorscale=colorscaled, 
        symbol=niwot_df_sub3['TypeClassSymbol'],
        showscale=False
        )
    ), row=2, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-22,8], row=2, col=1) 
fig.update_yaxes(title_text = namewind, range=[-.5,15], row=2, col=1) 

fig.add_trace(
    go.Scatter(
        x = niwot_df_sub3['temp'],
        y = niwot_df_sub3['shortwave'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=colorsnowunload,
        colorscale=colorscaled, 
        symbol=niwot_df_sub3['TypeClassSymbol'],
        showscale=False
        )
    ), row=3, col=1
)

fig.update_xaxes(title_text = nametemp, range=[-22,8], row=3, col=1) 
fig.update_yaxes(title_text = nameSW, range=[-50,1200], row=3, col=1)

fig.add_trace(
    go.Scatter(
        x = niwot_df_sub3['windspeed'],
        y = niwot_df_sub3['shortwave'],
        mode='markers',
        marker=dict(
        size=markersize,
        color=colorsnowunload, 
        colorscale=colorscaled, 
        symbol=niwot_df_sub3['TypeClassSymbol'],
        showscale=False,
        )
    ), row=3, col=2
)

fig.update_xaxes(title_text = namewind, range=[-.5,15], row=3, col=2) 
fig.update_yaxes(title_text = nameSW, range=[-50,1200], row=3, col=2)

fig.update_layout(title='', width=1000, height=1200, showlegend=False)
fig.show()

## Only negative temperatures...

In [26]:
niwot_df_sub3_negative = niwot_df_sub3.where(niwot_df_sub3.temp < -1).dropna(axis=0, how='any', subset=['CD']) # threshold at -1, negative only

In [37]:
fig = go.Figure(data=go.Scatter(
    x = niwot_df_sub3_negative['windspeed'],
    y = niwot_df_sub3_negative['shortwave'],
    mode='markers',
    marker=dict(
        size=10,
        color=niwot_df_sub3_negative['CD'], 
        colorscale= colorscaled,
        symbol=niwot_df_sub3_negative['TypeClassSymbol'],
        showscale=False
    )
))

fig.update_xaxes(title_text = namewind, range=[-.5,15])
fig.update_yaxes(title_text = nameSW, range=[-50,1200])

fig.update_layout(title='Niwot Ridge, Colorado, USA', width=800, height=700)

fig.show()

# 3D plot for fun...

In [24]:
# df = niwot_df_unload.copy(deep=True)

# # Some constatns 
# colorscaled = [[0., colorsnowunload], [1, colorsnow]]
# index_vals = df['CD']
# index_symbols = df['Typesymbol']

# fig = go.Figure()
# fig.add_trace(go.Splom(dimensions=[dict(label='Air Temperature (C)',     values=df['temp']),
#                                    dict(label='Wind Speed (m/s)', values=df['windspeed']),
#                                    dict(label='Shortwave (W/m2)', values=df['shortwave'])],
#                                    marker=dict(color=index_vals, colorscale=colorscaled, symbol=index_symbols, showscale=False, line_color='black', line_width=0.5, size=7),
#                                    showlegend=False, showupperhalf=False, diagonal_visible=False,
#                       ))

# fig.update_layout(title='Niwot Ridge, Colorado, USA', width=800, height=700, dragmode='select')

# fig.show()

In [25]:
# Niwot 3D
# create a new column for the partial and wind classification varible
niwot_df_3D = niwot_df_sub.copy(deep=True)
# niwot_df_3D['CD'] = niwot_df_3D['CD'].where(niwot_df_3D['CD'] == 0).dropna()
# niwot_df_3D['CD'].dropna(inplace=True)

# first, copy type into the new column
niwot_df_3D['TypeClassSymbol'] = niwot_df_3D['Typesymbol'].copy(deep=True) 

# replacing with the names?? because we need to for the 3D???
niwot_df_3D['TypeClassSymbol'].mask(niwot_df_3D['Typesymbol'] == 102, "diamond-open", inplace  = True) 
niwot_df_3D['TypeClassSymbol'].mask(niwot_df_3D['Typesymbol'] == 2, "diamond", inplace  = True) 

# IF WE WANT TO USE THE STARS AT ALL.....
# # then, change the values when Type == 1 AND Class == 17, then it's an open star for wind and partial unloading
# niwot_df_3D['TypeClassSymbol'].mask((niwot_df_3D['Typesymbol'] == 102) & (niwot_df_3D['Classsymbol'] == 17), "star-open", inplace  = True) # star-open (117)

# # last, change the values when Type == 2 AND Class == 17, then it's an closed star for wind and full unloading
# niwot_df_3D['TypeClassSymbol'].mask((niwot_df_3D['Typesymbol'] == 2) & (niwot_df_3D['Classsymbol'] == 17), "star", inplace  = True) # star (17)

# OPTION 1: last, fill all the snow staying with the circle
niwot_df_3D['TypeClassSymbol'] = niwot_df_3D['TypeClassSymbol'].fillna(128)  # empty circle
niwot_df_3D['TypeClassSymbol'].mask(niwot_df_3D['TypeClassSymbol'] == 128, "circle-open", inplace  = True) 

# OPTION 2: unless you do not want the snow staying to plot then remove everything else 
# niwot_df_sub3.dropna(subset=['TypeClassSymbol'], inplace=True)
# niwot_df_sub3.dropna(subset=['CD'], inplace=True)

In [92]:
fig = go.Figure(data=[go.Scatter3d(
    x=niwot_df_3D['temp'],
    y=niwot_df_3D['windspeed'],
    z=niwot_df_3D['shortwave'],
    mode='markers',
    marker=dict(
        size=5,
        color=niwot_df_3D['CD'],
        symbol=niwot_df_3D['TypeClassSymbol'],# trying this with the 3D dataframe and the string option for the symbols
        colorscale=colorscaled,   
        opacity=0.8,
    )
)])


fig.update_layout(
    scene = dict(
        xaxis = dict(title=nametemp, nticks=4), # nticks=4, range=[0,10]
        yaxis = dict(title=namewind), 
        zaxis = dict(title=nameSW)
    )
)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()